In [7]:
from pyscf import scf,gto
import basis_set_exchange as bse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pyscf.geomopt import optimize


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.append("../Alchemical_CPKS_perturbator")
from FcMole import *
from AP_class import APDFT_perturbator as AP
from AP_utils import DeltaV

In [6]:
basis_pcx2={"H":"pc-2",'B':bse.get_basis("pcX-2",fmt="nwchem",elements=[5])\
            ,'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])\
           ,'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])\
           ,'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])\
           ,'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])}

In [ ]:
# n2- co,bf
# CO- N2,BF

In [13]:
NN_nonopt=gto.M(atom="N 0 0 0; N 0 0 2.",basis=basis_pcx2,unit="au")
mf=scf.RKS(NN_nonopt)
mf.xc="PBE0"
NN=optimize(mf)

In [15]:
CO_nonopt=gto.M(atom="C 0 0 0; O 0 0 2.",basis=basis_pcx2,unit="au")
mf=scf.RKS(CO_nonopt)
mf.xc="PBE0"
CO=optimize(mf)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   O   0.000000   0.000000   1.058354    0.000000  0.000000  0.000000
converged SCF energy = -113.220868732787
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000     0.1984826316
1 O     0.0000000000    -0.0000000000    -0.1984873184
----------------------------------------------
cycle 1: E = -113.220868733  dE = -113.221  norm(grad) = 0.2807

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.079158    0.000000  0.000000 -0.079158
   O   0.000000   0.000000   1.137512    0.000000  0.000000  0.079158

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess de

In [16]:
BF_nonopt=gto.M(atom="B 0 0 0; F 0 0 2.1",basis=basis_pcx2,unit="au")
mf=scf.RKS(BF_nonopt)
mf.xc="PBE0"
BF=optimize(mf)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   B   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   F   0.000000   0.000000   1.111272    0.000000  0.000000  0.000000
converged SCF energy = -124.551872675851
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 B     0.0000000000     0.0000000000     0.2545019636
1 F    -0.0000000000    -0.0000000000    -0.2545427747
----------------------------------------------
cycle 1: E = -124.551872676  dE = -124.552  norm(grad) = 0.359949

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   B   0.000000   0.000000  -0.079377    0.000000  0.000000 -0.079377
   F   0.000000   0.000000   1.190649    0.000000  0.000000  0.079377

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess 

In [21]:
mf_NN=scf.RKS(NN)
mf_NN.xc="PBE0"
mf_NN.scf()
ap_nn=AP(mf_NN,sites=[0,1])
ap_nn.build_all()

converged SCF energy = -109.445602751152


In [20]:
mf_CO=scf.RKS(CO)
mf_CO.xc="PBE0"
mf_CO.scf()
ap_co=AP(mf_CO,sites=[0,1])
ap_co.build_all()

converged SCF energy = -113.232052500742


In [22]:
mf_BF=scf.RKS(BF)
mf_BF.xc="PBE0"
mf_BF.scf()
ap_bf=AP(mf_BF,sites=[0,1])
ap_bf.build_all()

converged SCF energy = -124.581813879953


In [29]:
APDFT1={}
APDFT2={}
APDFT3={}
half_appr={}
target_energies={}

mf=mf_NN
mol=NN
ap=ap_nn

E0=mf.e_tot
E0_nuc=mf.energy_nuc()

t=np.array([1,-1])
label="NN->CO"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t

t=np.array([2,-2])
label="NN->BF"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t


converged SCF energy = -113.223012850726
converged SCF energy = -110.390566498535
converged SCF energy = -124.512357756126
converged SCF energy = -113.223012850726


In [31]:
mf=mf_CO
mol=CO
ap=ap_co

E0=mf.e_tot
E0_nuc=mf.energy_nuc()


t=np.array([1,-1])
label="CO->NN"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t


t=np.array([-1,1])
label="CO->BF"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t


converged SCF energy = -109.44258746742
converged SCF energy = -110.391630694637
converged SCF energy = -124.550875658546
converged SCF energy = -117.956332123377


In [33]:
mf=mf_BF
mol=BF
ap=ap_bf

E0=mf.e_tot
E0_nuc=mf.energy_nuc()


t=np.array([1,-1])
label="BF->CO"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t


t=np.array([2,-2])
label="BF->NN"

dV=DeltaV(mol,[[0,1],t])
APDFT1[label]=ap.APDFT1(t)
APDFT2[label]=ap.APDFT2(t)
APDFT3[label]=ap.APDFT3(t)
tm=FcM_like(mol,fcs=[[0,1],t])
t_scf=scf.RKS(tm )
t_scf.xc="PBE0"
target_energies[label]=t_scf.scf(dm0=ap.dP_pred(t))
enuc_t=tm.energy_nuc()
hm=FcM_like(mol,fcs=[[0,1],t/2])
hm_scf=scf.RKS(hm)
hm_scf.xc="PBE0"
hm_scf.scf(dm0=ap.dP_pred(t/2))
half_appr[label]=E0+np.einsum("ij,ij",hm_scf.make_rdm1(),dV)-E0_nuc+enuc_t


converged SCF energy = -113.198589072831
converged SCF energy = -117.951841507704
converged SCF energy = -109.381527847298
converged SCF energy = -113.19858907283


In [35]:
df=pd.DataFrame()
df["APDFT1"]=pd.Series(APDFT1)
df["APDFT2"]=pd.Series(APDFT2)
df["APDFT3"]=pd.Series(APDFT3)
df["Target"]=pd.Series(target_energies)
df["Half aprrox"]=pd.Series(half_appr)
df

,APDFT1,APDFT2,APDFT3,Target,Half aprrox
NN->CO,-109.445603,-113.226244,-113.226244,-113.223013,-113.224665
NN->BF,-109.445603,-124.568169,-124.568169,-124.512358,-124.541883
CO->NN,-105.663884,-109.432637,-109.448296,-109.442587,-109.441002
CO->BF,-120.800221,-124.568974,-124.553315,-124.550876,-124.556470
BF->CO,-109.459916,-113.167301,-113.205194,-113.198589,-113.191729
BF->NN,-94.338018,-109.167556,-109.470706,-109.381528,-109.347134


In [37]:
error=pd.DataFrame()
error["APDFT1"] =(df["APDFT1"]-df.Target)*1000
error["APDFT2"] =(df["APDFT2"]-df.Target)*1000
error["APDFT3"] =(df["APDFT3"]-df.Target)*1000
error["Half aprrox"] =(df["Half aprrox"]-df.Target)*1000
error

,APDFT1,APDFT2,APDFT3,Half aprrox
NN->CO,3777.410100,-3.231375,-3.231375,-1.651653
NN->BF,15066.755006,-55.810895,-55.810895,-29.525550
CO->NN,3778.703444,9.950550,-5.708421,1.585568
CO->BF,3750.654680,-18.098214,-2.439242,-5.593977
BF->CO,3738.673120,31.288513,-6.605198,6.860285
BF->NN,15043.509822,213.971392,-89.178292,34.393438


In [38]:
df.to_pickle("NNCOBF.pkl")
error.to_pickle("NNCOBF_error.pkl")